In [117]:
# Dependencies and Setup
import csv
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import requests
import seaborn as sns
import time



In [118]:
dow_jones_df= pd.read_csv('/Users/dianeshomefolder/Desktop/Dow Jones/dow_jones2.csv')
#dow_jones_df.set_index('timestamp')
dow_jones_df.head()


,timestamp,open,high,low,close,volume
0,2019-09-27,26987.2598,27012.5391,26715.8203,26820.2500,217780000
1,2019-09-26,27004.1094,27015.0703,26803.8398,26891.1191,229180000
2,2019-09-25,26866.7109,27016.5605,26755.8594,26970.7109,237220000
3,2019-09-24,27034.0703,27079.6797,26704.9609,26807.7695,301750000
4,2019-09-23,26851.4492,27011.0703,26831.3398,26949.9902,204240000


In [119]:
dow_jones_close = dow_jones_df
dow_jones_close['close-open'] = dow_jones_df['close'] - dow_jones_close['open']
dow_jones_close.head()


,timestamp,open,high,low,close,volume,close-open
0,2019-09-27,26987.2598,27012.5391,26715.8203,26820.2500,217780000,-167.0098
1,2019-09-26,27004.1094,27015.0703,26803.8398,26891.1191,229180000,-112.9903
2,2019-09-25,26866.7109,27016.5605,26755.8594,26970.7109,237220000,104.0000
3,2019-09-24,27034.0703,27079.6797,26704.9609,26807.7695,301750000,-226.3008
4,2019-09-23,26851.4492,27011.0703,26831.3398,26949.9902,204240000,98.5410


In [120]:
dow_jones_close['close-open'] = [float(x) for x in dow_jones_close['close-open']]
dow_jones_close.head()

,timestamp,open,high,low,close,volume,close-open
0,2019-09-27,26987.2598,27012.5391,26715.8203,26820.2500,217780000,-167.0098
1,2019-09-26,27004.1094,27015.0703,26803.8398,26891.1191,229180000,-112.9903
2,2019-09-25,26866.7109,27016.5605,26755.8594,26970.7109,237220000,104.0000
3,2019-09-24,27034.0703,27079.6797,26704.9609,26807.7695,301750000,-226.3008
4,2019-09-23,26851.4492,27011.0703,26831.3398,26949.9902,204240000,98.5410


In [102]:
#check that close-open are now float numbers vs. string
a = dow_jones_close['close-open'][0] - dow_jones_close['close-open'][1]
print(a)

-54.019499999998516


In [114]:
#dow_jones_close['timestamp'] = [x.replace('-', '') for x in dow_jones_close['timestamp']]
#dow_jones_close['timestamp'] = [int(x) for x in dow_jones_close['timestamp']]
#dow_jones_close.head()

In [121]:
dow_jones_close_no_index = dow_jones_close.set_index('timestamp')
dow_jones_close_no_index.head()


,open,high,low,close,volume,close-open
timestamp,,,,,,
2019-09-27,26987.2598,27012.5391,26715.8203,26820.2500,217780000,-167.0098
2019-09-26,27004.1094,27015.0703,26803.8398,26891.1191,229180000,-112.9903
2019-09-25,26866.7109,27016.5605,26755.8594,26970.7109,237220000,104.0000
2019-09-24,27034.0703,27079.6797,26704.9609,26807.7695,301750000,-226.3008
2019-09-23,26851.4492,27011.0703,26831.3398,26949.9902,204240000,98.5410


In [109]:
#check that close-open are now float numbers vs. string
b = dow_jones_close['timestamp'][0] - dow_jones_close['timestamp'][1]
print(b)

1


In [123]:
trump_negative = dow_jones_close_no_index.lt(0).sum().sum() 
print(trump_negative)

2312


In [122]:
date_trump_election = '20161108'
date_trump_inaug = '20170120'

count_positive = 0
count_negative = 0

for timestamp in dow_jones_close_no_index.loc[date_trump_election:date_trump_inaug]:
    if dow_jones_close_no_index['close-open'] >= 0:
        count_positive += 1
        print(count_positive)
    elif dow_jones_close_no_index['close-open'] < 0:
        count_negative += 1
        print(count_negative)
print(count_positive)
                
           

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
date_bins = []

In [ ]:
bin_labels = []